In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numerate import NumericalAdvectionEquationFluxLimiter
from numerate.limiters import *
from numerate.functions import *
from numerate.verification import *

In [ ]:
f = lambda x : gaussian(x, b = 0.05, c=0.3) + tophat(x, b=0.2, c=0.7)
g = lambda x : tophat(x, b=0.2, c=0.7)
h = lambda x : gaussian(x, b=0.05, c=0.3)

In [ ]:
a = 1
x0 = 0
x1 = 1
xs = 300
revolutions = 3
ts = 500 * revolutions
func = f
p = 1

In [ ]:
def flux(phi, *, save=True, **kwargs):
    eq = NumericalAdvectionEquationFluxLimiter(
    a,
    func,
    x0=x0,
    x1=x1,
    xs=xs,
    revolutions=revolutions,
    ts=ts,
    phi=phi,
    epsilon=1e-16,
    **kwargs,
    )
    solution = eq.solve()

    print("Is TVD:", is_tvd(solution.toarray()))
    for i in range(revolutions+1):
        print(f"TV at t = {i}:", total_variation(solution[:, eq.get_temporal_index(i)].toarray()))
    
    eq.plot(solution)
    
    fig, ax = plt.subplots()
    
    r = np.linspace(-1, 5, 3000)
    
    func1 = np.piecewise(
        r,
        [np.logical_and(r > 0, r < 0.5), r >= 0.5],
        [lambda x: 2 * x, lax_wendroff]
    )
    
    func2 = np.piecewise(
        r,
        [np.logical_and(r > 0, r < 2), r >= 2],
        [beam_warming, lambda x: 2]
    )
    
    ax.plot(r, phi(r, **kwargs))
    ax.fill_between(
        r, 
        func1,
        func2,
        where=r > 0,
        color='black',
        alpha=0.05,
    )
    ax.set_xlabel(r'$r$')
    ax.set_ylabel(r'$\phi\left(r\right)$', loc='top')
    ax.set_xlim([-1, 5])
    ax.set_ylim([-1, 3])
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
    
    ax.spines['left'].set_position('zero')
    ax.spines['right'].set_color('none')

    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_color('none')
    
    plt.show()

In [ ]:
flux(sweby, beta=1.0)

In [ ]:
flux(sweby, beta=2.0)

In [ ]:
flux(sweby, beta=1.5)

In [ ]:
flux(mc)

In [ ]:
flux(van_leer)